In [11]:
###Import
import sys

#import geobayes_simple as gs
from matplotlib import pyplot as plt
from IPython.core.display import Image

%matplotlib inline
import pymc as pm
from pymc.Matplot import plot
import numpy as np
from IPython.core.pylabtools import figsize
figsize(15, 6)
import seaborn
seaborn.set_context(font_scale=2)
seaborn.set_style("white")

from pymc import graph

import scipy.optimize as sop
import scipy.stats as stats
import matplotlib.mlab as mlab

from scipy.signal import argrelextrema

In [12]:
fault_block = np.load('block_faults.npy') #1=hanging wall; 0=footwall
lith_block = np.load('block_lith.npy')
non_res = np.load('Non_res.npy')
res = np.load('res.npy')
seal = np.load('seal.npy')
res2 = np.load('sec_res.npy')

In [16]:
#spill_z = 21.164197799910983
#leak_z = 20.293855462612264
resolution = 50
model_size = 2000
scale_factor = (model_size/resolution) #original grid in [m]/grid resolution --> what if model not cubic?

In [5]:
#finding the spill point 
def spill_point(res_top):
    mini = argrelextrema(res[:,2], np.less, order=10)
    minima = np.array(list(zip(np.take(res[:,1], mini[0]), np.take(res[:,0], mini[0]), np.take(res[:,2], mini[0]))))
    fault_thresh = minima[:,1] > 18 #taking only relevant side by setting a threshold visually estimated, best: 20
    min_corr_side = minima[fault_thresh]
    spill_pos = np.array(np.argmax(min_corr_side[:,2]))
    spill_z = np.take(min_corr_side[:,2], spill_pos)
    spill_point = np.array([np.take(min_corr_side[:,1], spill_pos), np.take(min_corr_side[:,0], spill_pos), np.take(min_corr_side[:,2], spill_pos)])
    return(spill_z)

In [6]:
def leak_point(res_top):
    counter = 0

    xvals = []
    yvals = []
    zvals = []
    leak_min = np.empty([3,], dtype=int)
    
    while counter < resolution:
        for e in res:
            if e[1] == counter:
                xvals.append(e[0])
                yvals.append(e[1])
                zvals.append(e[2])
              
        zvals = np.array(zvals)
        min_pos = argrelextrema(zvals, np.less, order=10)
        yvals = np.array(yvals)
        xvals = np.array(xvals)
        ypos = yvals[min_pos]
        xpos = xvals[min_pos]
        mins = zvals[min_pos]
        
        np.append(leak_min, mins)
        mins_pos = np.array(list(zip(xpos, ypos, mins)))
        leak_min = np.vstack((leak_min, mins_pos))

        xvals = []
        yvals = []
        zvals = []
    
        counter += 1
        
    leak_min = np.delete(leak_min, 0, 0)
    
    near_fault_thresh = leak_min[:,0] < 30 #taking only relevant side by setting a threshold visually estimated, best: 20
    leak_line = leak_min[near_fault_thresh]
    
    leak_pos = np.array(np.argmax(leak_line[:,2]))
    leak_z = np.take(leak_line[:,2], leak_pos)

    leak_p = np.array([np.take(leak_line[:,1], leak_pos), np.take(leak_line[:,0], leak_pos), np.take(leak_line[:,2], leak_pos)])
    
    return(leak_z)

In [7]:
def max_res_vol(lith, fault, res_top):
    #calculate spill point
    spill_z = spill_point(res_top)
    #calculate leak point
    leak_z = leak_point(res_top)
    
    #check for "down-to" z horizon, maximum depth of reservoir
    max_z = np.max([spill_z, leak_z])
    
    if max_z == spill_z:
        print("Down to spill.")
    else:
        print("Down to leak.")
    
    #reducing lithology to reservoir and non.reservoir
    lith[fault.astype(bool)] = 6
    lith = lith.reshape(50,50,50)
    lith[:,:,:max_z] = 6
    lith = lith.reshape(125000,)
    
    #counting reservoir cells
    vol_cells = 0
    for i in range(lith.shape[0]):
        if lith[i] != 5:
            lith[i] = 6
        else:
            vol_cells += 1
    
    #calulate volume from cells
    res_vol = ((scale_factor)**3) * vol_cells
    
    #return the maximum reservoir volume
    return(res_vol)
    

In [8]:
print("Maximum reservoir volume: %s m³" % max_res_vol(lith_block, fault_block, res))

Down to spill.
Maximum reservoir volume: 39040000.0 m³


/home/bl9/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [9]:
def recov_res(vol, porosity, sat_wat, form_f, RF):
    ooip = (vol*porosity*(1-sat_wat))/(form_f)
    recov = ooip * RF
    return(recov)

In [10]:
recoverable_reserves = recov_res(vol=max_res_vol(lith_block, fault_block, res), porosity=0.3, sat_wat = 0.5, form_f = 1.5, RF = 0.9)
print("Maximum recoverable reserves: %s m³" % recoverable_reserves)

Down to spill.
Maximum recoverable reserves: 3513600.0 m³


/home/bl9/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
